# New GDP Real-Time Dataset

**Author:** Jason Cruz  
**Last updated:** 08/13/2025  
**Python version:** 3.12  
**Project:** Rationality and Nowcasting on Peruvian GDP Revisions  

---
## 📌 Summary
This notebook documents the step-by-step **construction of datasets** for analyzing **Peruvian GDP revisions** from 2013–2024.  
It covers:
1. **Data acquisition** from the Central Reserve Bank of Peru's Weekly Reports (PDF).
2. **Data cleaning** and extraction of GDP tables.
3. **Creation of real-time GDP vintages**.
4. **Preparation of the final revisions dataset**.
5. **Export to SQL** for further analysis.

🌐 **Main Data Source:** [BCRP Weekly Report](https://www.bcrp.gob.pe/publicaciones/nota-semanal.html) (📰 WR, from here on)  
Any questions or issues regarding the coding, please email [Jason 📨](mailto:jj.cruza@alum.up.edu.pe)  

---

## 🛠️ Libraries

If you don't have the libraries below, please use the following code (as example) to install the required libraries.

In [ ]:
#!pip install os # Comment this code with "#" if you have already installed this library.

Check out Python information

In [1]:
import sys
import platform

print("🐍 Python Information")
print(f"  Version  : {sys.version.split()[0]}")
print(f"  Compiler : {platform.python_compiler()}")
print(f"  Build    : {platform.python_build()}")
print(f"  OS       : {platform.system()} {platform.release()}")

🐍 Python Information
  Version  : 3.12.1
  Compiler : MSC v.1916 64 bit (AMD64)
  Build    : ('main', 'Jan 19 2024 15:44:08')
  OS       : Windows 10


**Import helper functions**

> ⚠️ Please, check the script `new_gdp_datasets_functions.py` which contains all the functions required by this _jupyter notebook_. The functions there are ordered according to the sections of this jupyter notebok.

In [2]:
from new_gdp_datasets_functions import *

pygame 2.5.2 (SDL 2.28.3, Python 3.12.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


## ⚙️ Initial set-up

Before preprocessing new GDP releases data, we will:

* **Create necessary folders** for storing inputs, outputs, logs, and screenshots.
* **Connect to the PostgreSQL database** containing GDP revisions datasets.
* **Import helper functions** from `new_gdp_datasets_functions.py`.

**Create necessary folders**

In [3]:
from pathlib import Path

PROJECT_ROOT = Path.cwd()
user_input = input("Enter relative path (default='.'): ").strip() or "."
target_path = (PROJECT_ROOT / user_input).resolve()
target_path.mkdir(parents=True, exist_ok=True)
print(f"📂 Using path: {target_path}")


Enter relative path (default='.'):  .


📂 Using path: C:\Users\Jason Cruz\OneDrive\Documentos\RA\CIUP\GDP Revisions\GitHub\peru_gdp_revisions\gdp_revisions_datasets


**Connect to the PostgreSQL database**

The following function will establish a connection to the `gdp_revisions_datasets` database in `PostgreSQL`. The **input data** used in this jupyter notebook will be loaded from this `PostgreSQL` database, and similarly, all **output data** generated by this jupyter notebook will be stored in that database. Ensure that you set the necessary parameters to access the server once you have obtained the required permissions.

> 💡 **Tip:** To request permissions, please email [Jason 📨](mailto:jj.cruza@alum.up.edu.pe)  
> ⚠️ **Warning:** Make sure you have set your SQL credentials as environment variables before proceeding.  

In [ ]:
from sqlalchemy import create_engine
import os

In [ ]:
def create_sqlalchemy_engine(database="gdp_revisions_datasets", port=5432):
    """
    Create an SQLAlchemy engine to connect to the PostgreSQL database.
    
    Environment Variables Required:
        CIUP_SQL_USER: SQL username
        CIUP_SQL_PASS: SQL password
        CIUP_SQL_HOST: SQL host address

    Args:
        database (str): Name of the database. Default is 'gdp_revisions_datasets'.
        port (int): Port number. Default is 5432.

    Returns:
        engine (sqlalchemy.engine.Engine): SQLAlchemy engine object.
    
    Raises:
        ValueError: If required environment variables are missing.

    Example:
        engine = create_sqlalchemy_engine()
    """
    user = os.environ.get('CIUP_SQL_USER')
    password = os.environ.get('CIUP_SQL_PASS')
    host = os.environ.get('CIUP_SQL_HOST')

    if not all([host, user, password]):
        raise ValueError("❌ Missing environment variables: CIUP_SQL_HOST, CIUP_SQL_USER, CIUP_SQL_PASS")

    connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"
    engine = create_engine(connection_string)

    print(f"🔗 Connected to PostgreSQL database: {database} at {host}:{port}")
    return engine

In [ ]:
engine = create_sqlalchemy_engine()

## 1. PDF Downloader

Our main source for data collection is the [BCRP Weekly Report](https://www.bcrp.gob.pe/publicaciones/nota-semanal.html). The weekly report is a periodic (weekly) publication of the BCRP in compliance with article 84 of the Peruvian Constitution and articles 2 and 74 of the BCRP's organic law, which include, among its functions, the periodic publication of the main national macroeconomic statistics.
    
Our project requires the publication of **two tables**: the table of monthly growth rates of real GDP (12-month percentage changes), and the table of quarterly (annual) growth rates of real GDP. These tables are referred to as **Table 1** and **Table 2**, respectively, throughout this jupyter notebook.

### Scraper bot

This section automates the download of the **BCRP Weekly Report PDFs** directly from the official BCRP website.

**What it does:**
1. Opens the official BCRP Weekly Report page.
2. Finds and collects all PDF links.
3. Downloads them in chronological order (oldest to newest).
4. Optionally plays a notification sound every N downloads.
5. Organizes downloaded PDFs into year-based folders.

> 💡 If a CAPTCHA appears, solve it manually in the browser window and re-run the cell.

> 🔁 This script uses webdriver-manager to automatically handle browser drivers (default: Chrome), so you DO NOT need to manually download ChromeDriver, GeckoDriver, etc. If you want to change browser for your replication, modify the 'browser' parameter in init_driver().

> 🎵 Place your own MP3 file in `alert_track` folder for download notifications. Recommended free sources (CC0/public domain):
>  - Pixabay Audio: https://pixabay.com/music/
>  - FreeSound: https://freesound.org/
>  - FreePD: https://freepd.com/

In [9]:
# Define base folder for saving all digital PDFs
pdf_folder = 'pdf'

# Define subfolder for saving the original PDFs as downloaded from the BCRP website
raw_pdf_subfolder = os.path.join(pdf_folder, 'raw')

# Define subfolder for saving reduced PDFs containing only selected pages with GDP growth tables (monthly, quarterly, and annual frequencies)
input_pdf_subfolder = os.path.join(pdf_folder, 'input')

# Define folder for saving .txt files with download and dataframe record
record_folder = 'record'

# Define folder for saving warning bells. This is for download notifications (see section 1).
alert_track_folder = 'alert_track'

# Create all required folders (if they do not already exist) and confirm creation
for folder in [pdf_folder, raw_pdf_subfolder, input_pdf_subfolder, record_folder, alert_track_folder]:
    os.makedirs(folder, exist_ok=True)
    print(f"📂 {folder} created")

📂 pdf created
📂 pdf\raw created
📂 pdf\input created
📂 record created
📂 alert_track created


In [11]:
# Run the function to start the scraper bot
pdf_downloader(
    bcrp_url = "https://www.bcrp.gob.pe/publicaciones/nota-semanal.html",
    raw_pdf_folder = raw_pdf_subfolder,
    download_record_folder = record_folder,
    download_record_txt = '1_downloaded_pdf.txt',
    alert_track_folder = alert_track_folder,
    max_downloads = 60,
    downloads_per_batch = 6, 
    headless = False 
)


📥 Starting PDF downloader for BCRP WR...

🌐 BCRP site opened successfully.
🔎 Found 154 WR blocks on page (one per month).

1. ✅ Downloaded: ns-04-2024.pdf
⏳ Waiting 9.31 seconds...
2. ✅ Downloaded: ns-08-2024.pdf
⏳ Waiting 8.99 seconds...
3. ✅ Downloaded: ns-11-2024.pdf
⏳ Waiting 9.12 seconds...
4. ✅ Downloaded: ns-15-2024.pdf
⏳ Waiting 7.25 seconds...
5. ✅ Downloaded: ns-19-2024.pdf
⏳ Waiting 7.55 seconds...
6. ✅ Downloaded: ns-23-2024.pdf
⏳ Waiting 9.18 seconds...
7. ✅ Downloaded: ns-27-2024.pdf
⏳ Waiting 9.32 seconds...
8. ✅ Downloaded: ns-31-2024.pdf
⏳ Waiting 5.40 seconds...
9. ✅ Downloaded: ns-35-2024.pdf
⏳ Waiting 6.95 seconds...
10. ✅ Downloaded: ns-39-2024.pdf
⏳ Waiting 5.56 seconds...
11. ✅ Downloaded: ns-43-2024.pdf
⏳ Waiting 6.23 seconds...
12. ✅ Downloaded: ns-47-2024.pdf


⏸️ Continue? (y = yes, any other key = stop):  y


⏳ Waiting 6.50 seconds...
13. ✅ Downloaded: ns-04-2025.pdf
⏳ Waiting 7.05 seconds...
14. ✅ Downloaded: ns-08-2025.pdf
⏳ Waiting 6.53 seconds...
15. ✅ Downloaded: ns-11-2025.pdf
⏳ Waiting 9.75 seconds...
16. ✅ Downloaded: ns-14-2025.pdf
⏳ Waiting 5.34 seconds...
17. ✅ Downloaded: ns-18-2025.pdf
⏳ Waiting 8.65 seconds...
18. ✅ Downloaded: ns-22-2025.pdf
⏳ Waiting 9.55 seconds...
19. ✅ Downloaded: ns-26-2025.pdf
⏳ Waiting 9.61 seconds...
20. ✅ Downloaded: ns-30-2025.pdf
⏳ Waiting 9.12 seconds...
21. ✅ Downloaded: ns-34-2025.pdf
⏳ Waiting 7.39 seconds...
22. ✅ Downloaded: ns-36-2025.pdf
⏳ Waiting 5.08 seconds...

👋 Browser closed.

📊 Summary:

🔗 Total monthly links kept: 154
🗂️ 132 already downloaded PDFs were skipped.
➕ Newly downloaded: 22
⏱️ 379 seconds


Probably the 📰 WR were downloaded in a single folder, but we would like the WR to be sorted by years. The following code sorts the PDFs into subfolders (years) for us by placing each WR according to the year of its publication. This happens in the **"blink of an eye"**.

Check your raw_pdf_subfolder out, every PDF should be placed in a year folder.

In [7]:
# Get the list of files in the directory
files = os.listdir(raw_pdf_subfolder)

# Call the function to organize files
organize_files_by_year(raw_pdf_subfolder)

⚠️ No 4-digit year detected in filename: _quarantine


# WR-08-2017

This  is crucial for the upcoming steps, specially for the section 3, cleansing. If -in the future- you enconuter some issues by executing cleaing it is likely to atributte to the pdf nature. IN that case, you can return to this code to replace defectiv pdfs for those convinient ones

Don't worry about it...

Tú puedes hacer lo mismo si te enfrentas a un inconveniente similar. Incluso puedes descargar los casos excepecionales de WR de un mismo mes y reemplazar los defectuosos.

In [ ]:
# Replace specific defective PDFs (friendly outputs with icons)
replace_ns_pdfs(
    items=[
        ("2017", "ns-08-2017.pdf", "ns-07-2017"), # Enter the year (folder) that contains the defective PDF, the defective PDF, and the new chosen PDF 
        ("2019", "ns-23-2019.pdf", "ns-22-2019"), # The same one above
    ],
    root_folder=input_pdf_subfolder, # base folder with /2017, /2019, ...
    record_folder=record_folder, # folder with new_downloaded_pdfs.txt
    download_record_txt = 'new_downloaded_pdfs.txt',
    quarantine=os.path.join(input_pdf_subfolder, "_quarantine")  # set to None to delete instead
)

## 2. Generate PDF input with key tables

Now that we have downloaded the 📰 WR from the Central Bank, we should know that each of these files has more than 100 pages, but not all of them contain the information required for this project.

All we really want is a couple of pages from each 📰 WR, one for **Table 1** (monthly real GDP growth) and one for **Table 2** (annual and quarterly real GDP growth). The code below is executed to maintain the **two key pages** with both tables of each PDF plus the cover page that contains the information that helps us identify one 📰 WR from another such as its date of publication and serial number.

_quarentine will be discard of the input PDF generator

In [ ]:
# Run the function to generate trimmed PDFs for input
input_pdfs_generator(
    raw_pdf_folder = raw_pdf_subfolder,
    input_pdf_folder = input_pdf_subfolder,
    input_pdf_record_folder = record_folder,
    input_pdf_record_txt = 'new_generated_input_pdfs.txt',
    keywords = ["ECONOMIC SECTORS"]
)

Again, probably the WR (PDF files, now of few pages) were stored in disorder in the `input_pdf_folder` folder. The following code sorts the PDFs into subfolders (years) by placing each WR (which now includes only the key tables) according to the year of its publication. This happens in the **"blink of an eye"**.  

In [ ]:
# Get the list of files in the directory
files = os.listdir(input_pdf_subfolder)

# Call the function to organize files
organize_files_by_year(input_pdf_subfolder)

## 3. Data cleaning

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px">
<p>     
Since we already have the PDFs <span style="font-size: 24px;">&#128462;</span> with just the tables required for this project, we can start extracting them. Then we can proceed with data cleaning.
</p>  
<div/>

### 3.2 Extracting tables and data cleanup

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px">
<p>     
The main library used for extracting tables from PDFs <span style="font-size: 24px;">&#128462;</span> is <code>pdfplumber</code>. You can review the official documentation by clicking <a href="https://github.com/jsvine/pdfplumber" style="color: rgb(0, 153, 123); font-size: 16px;">here</a>.
</p>
    
<p>     
    The functions in <b>Section 3</b> of the <code>"new_gdp_datasets_functions.py"</code> script were built to deal with each of these issues. An interesting exercise is to compare the original tables (the ones in the PDF <span style="font-size: 24px;">&#128462;</span>) and the cleaned tables (by the cleanup codes below). Thus, the cleanup codes for <a href="#3-2-1" style="color: rgb(0, 153, 123); font-size: 16px;">Table 1</a> and <a href="#3-2-1" style="color: rgb(0, 153, 123); font-size: 16px;">Table 2</a> generates two dictionaries, the first one stores the raw tables; that is, the original tables from the PDF <span style="font-size: 24px;">&#128462;</span> extracted by the <code>pdfplumber</code> library, while the second dictionary stores the fully cleaned tables.
</p>
<div/>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px">
    The code iterates through each PDF <span style="font-size: 24px;">&#128462;</span> and extracts the two required tables from each. The extracted information is then transformed into dataframes and the columns and values are cleaned up to conform to Python conventions (pythonic).
    <div/>

<h3><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.2.1.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    <span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">Table 1.</span> Extraction and cleaning of data from tables on monthly real GDP growth rates.
    </span>
    </h3>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px">
<p>     
The basic criterion to start extracting tables is to use keywords (sufficient condition). I mean, tables containing the following keywords meet the requirements to be extracted.
</p>
<div/>

<div style="text-align: left;">
    <span style="font-size: 24px; color: rgb(255, 32, 78); font-weight: bold;">&#9888;</span>
    <span style="font-family: PT Serif Pro Book; color: black; font-size: 16px;">
        Please check that the flat file <b>"ns_dates.csv"</b> is updated with the dates, years and ids for the newly downloaded PDF <span style="font-size: 24px;">&#128462;</span> (WR). That file is located in the <b>"ns_dates"</b> folder and is uploaded to SQL from the jupyeter notebook <code>aux_files_to_sql.ipynb</code>
    </span>
</div>

Si por alguna razón ejecutas el código de la sección 3 y no continuas ejecutando la sección subsecuente, puedes estar tranquilo de que un registro los guardó. La próxima vez que visite este script basta con empezar desde esta sección 3 (eliminando el txt) para generar los dataframes que no se guardaron en ningun lado, estos son insumos esenciales para la sección 4. Alternativamente puede guardar todos los dataframes generados en una carpeta como respaldo y empezar desde la sección 4 cargándolos.

# Section 3 — Cleaning pipelines (Table 1 & Table 2)

### Functions
- `table_1_cleaner(...)` → cleans **Table 1 (monthly)** pages and returns two dicts:
  - `raw_tables_dict_1`: raw tables exactly as extracted from PDFs, keyed as `ns_xx_yyyy_1`.
  - `new_dataframes_dict_1`: cleaned tables ready for downstream steps, keyed as `ns_xx_yyyy_1`.

- `table_2_cleaner(...)` → cleans **Table 2 (quarterly/annual)** pages and returns:
  - `raw_tables_dict_2`: raw tables, keyed as `ns_xx_yyyy_2`.
  - `new_dataframes_dict_2`: cleaned tables, keyed as `ns_xx_yyyy_2`.

Both functions:
- **skip** year folder `_quarantine`
- maintain a **record txt** (chronologically sorted: year → issue)  
- show **Jupyter progress bars** (magenta = active, blue = finished)
- write a **log file**:
  - Table 1 → `logs/3_cleaner_1.log`
  - Table 2 → `logs/3_cleaner_2.log`

### Arguments
- `input_pdf_folder: str`  
  Root containing year subfolders with input PDFs (e.g., `input_pdf_subfolder/2017/ns-07-2017.pdf`).

- `record_folder: str`  
  Folder where the record txt is stored (e.g., `record/`).

- `record_txt: str` *(optional)*  
  Record filename. Defaults:
  - Table 1 → `new_generated_dataframes_1.txt`
  - Table 2 → `new_generated_dataframes_2.txt`

- `log_folder: str` *(optional, default `logs`)*  
  Where the `.log` files are written.

- `log_txt: str` *(optional)*  
  Log filename. Defaults:
  - Table 1 → `3_cleaner_1.log`
  - Table 2 → `3_cleaner_2.log`

- `persist: bool` *(optional, default `False`)*  
  If `True`, save cleaned tables to disk and update a manifest.  
  If `False`, nothing is saved (keeps repo light and re-runnable).

- `persist_folder: str | None` *(optional)*  
  Base folder for persisted outputs (default: `./data/clean`).  
  Layout when `persist=True`:
    data/clean/
    table_1/
    manifest.csv
    2017/
    ns-07-2017.parquet (or .csv if Parquet engine unavailable)
    table_2/
    manifest.csv
    2017/
    ns-07-2017.parquet

- `pipeline_version: str` *(optional, default `"s3.0.0"`)*  
Version tag recorded in `manifest.csv` for cache/audit. Bump it when the cleaning logic changes.

### Typical calls

```python
# Table 1 (monthly)
raw_1, clean_1 = table_1_cleaner(
  input_pdf_folder=input_pdf_subfolder,
  record_folder=record_folder,
  persist=True,                           # turn on checkpointing (Parquet/CSV + manifest)
  persist_folder=clean_data,              # e.g., os.path.join(project_root, "data", "clean")
  pipeline_version="s3.0.0"
)

# Table 2 (quarterly/annual)
raw_2, clean_2 = table_2_cleaner(
  input_pdf_folder=input_pdf_subfolder,
  record_folder=record_folder,
  persist=True,
  persist_folder=clean_data,
  pipeline_version="s3.0.0"
)



If you want the runners to *also* write the cleaned dicts out to a single combined Parquet/CSV per table (alongside the per-WR files), I can add that as an optional flag (`persist_combined=True`) without changing the defaults.


# If you will run until this section and you are planning to go back and retake from section 4, enter "True"

In [ ]:
data = 'data'

clean_data = os.path.join("data", "clean")  # choose your path

raw_1, clean_1, vintages_1 = table_1_cleaner(
    input_pdf_folder=input_pdf_subfolder,
    record_folder=record_folder,
    persist=True,
    persist_folder=clean_data,
    pipeline_version="s3.0.0",
)


In [ ]:
raw_1.keys()

In [ ]:
clean_1.keys()

In [ ]:
vintages_1.keys()

In [ ]:
raw_1['ns_04_2022_1']

In [ ]:
clean_1['ns_04_2022_1']

In [ ]:
vintages_1['ns_04_2022_1']

# Checking the cleaning version out

In [ ]:
df100 = vintages_1["ns_04_2022_1"]
print(df100.attrs)
# {'pipeline_version': 's3.0.0'}


In [ ]:
vintages_1["ns_04_2022_1"].attrs

<h3><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.2.2.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    <span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">Table 2.</span> Extraction and cleaning of data from tables on quarterly and annual real GDP growth rates.
    </span>
    </h3>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px">
<p>     
The basic criterion to start extracting tables is to use keywords (sufficient condition). I mean, tables containing the following keywords meet the requirements to be extracted.
</p>
<div/>

<div style="text-align: left;">
    <span style="font-size: 24px; color: rgb(255, 32, 78); font-weight: bold;">&#9888;</span>
    <span style="font-family: PT Serif Pro Book; color: black; font-size: 16px;">
        Please check that the flat file <b>"ns_dates.csv"</b> is updated with the dates, years and ids for the newly downloaded PDF <span style="font-size: 24px;">&#128462;</span> (WR). That file is located in the <code>ns_dates</code> folder and is uploaded to SQL from the jupyeter notebook <code>aux_files_to_sql.ipynb</code>
    </span>
</div>

In [ ]:
# Table 2 (quarterly/annual)
raw_2, clean_2 = table_2_cleaner(
  input_pdf_folder=input_pdf_subfolder,
  record_folder=record_folder,
  persist=True,
  persist_folder=clean_data,
  pipeline_version="s3.0.0"
)

In [ ]:
raw_2['ns_04_2022_2']

In [ ]:
clean_2['ns_04_2022_2']

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">4.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">GDP Real-Time dataset</span></h1>

In [ ]:
import re
import os
import pandas as pd

def build_month_order_map(year_folder: str) -> dict:
    """
    Creates a {filename:month_order} dictionary mapping based on the numeric value (dd)
    of the ns-dd-yyyy pattern of each PDF file in the year_folder. This ensures a correct
    chronological order, regardless of irregularities in the numbering.
    """
    files = [f for f in os.listdir(year_folder) if f.endswith(".pdf")]
    # Extract the numeric middle part (dd)
    file_nums = [(f, int(re.search(r'ns-(\d{2})-\d{4}', f).group(1))) for f in files]
    # Sort by dd and assign month order 1..12
    sorted_files = sorted(file_nums, key=lambda x: x[1])
    return {fname: i + 1 for i, (fname, _) in enumerate(sorted_files)}


def prepare_table_1(filepath: str, month_order_map: dict) -> pd.DataFrame:
    """
    
    """
    df = pd.read_csv(filepath)
    filename = os.path.basename(filepath)

    # 1. Assign month order (already extracted from all filenames)
    df["month"] = build_month_order_map.get(filename)

    # 2. Drop unused columns
    df = df.drop(columns=["wr", "sectores_economicos"], errors="ignore")

    # 3. Create short economic sector labels
    SECTOR_MAP = {
        "agriculture and livestock": "agriculture",
        "fishing": "fishing",
        "mining and fuel": "mining",
        "manufacturing": "manufacturing",
        "electricity and water": "electricity",
        "construction": "construction",
        "commerce": "commerce",
        "other services": "services",
        "gdp": "gdp"
    }
    df["economic_sector"] = df["economic_sectors"].map(SECTOR_MAP)

    # 4. Filter valid sectors
    df = df[df["economic_sector"].notna()].copy()

    # 5. Create 'vintage_id'
    df["vintage_id"] = (
        df["economic_sector"] + "_" + df["year"].astype(str) + "_" + df["month"].astype(str)
    )

    # 6. Keep only yyyy_mmm columns
    pattern = re.compile(r"^\d{4}_(ene|feb|mar|abr|may|jun|jul|ago|sep|oct|nov|dic)$")
    cols_to_keep = ["vintage_id"] + [c for c in df.columns if pattern.match(c)]
    df = df[cols_to_keep]

    # 7. Transpose and convert to tidy long format
    df_t = df.set_index("vintage_id").T.reset_index().rename(columns={"index": "target_period"})
    
    # Mapping from Spanish month abbreviations to month numbers
    month_map = {
        "ene": "01", "feb": "02", "mar": "03", "abr": "04", "may": "05", "jun": "06",
        "jul": "07", "ago": "08", "sep": "09", "oct": "10", "nov": "11", "dic": "12"
    }
    
    # Convert to 'yyyymx' format (string)
    df_t["target_period"] = (
        df_t["target_period"]
        .str.replace(
            r"(\d{4})_(\w{3})",
            lambda m: f"{m.group(1)}m{int(month_map.get(m.group(2).lower(), '1'))}",
            regex=True,
        )
    )

    return df_t

In [ ]:
def clean_table1_year(folder_path: str, year: str) -> dict:
    """Clean all monthly files for a given year and return dict of cleaned DataFrames."""
    year_folder = os.path.join(folder_path, year)
    month_order_map = build_month_order_map(year_folder)
    cleaned = {}
    for file in sorted(month_order_map.keys()):
        cleaned[file] = clean_table_1_file(os.path.join(year_folder, file), month_order_map)
    return cleaned

In [ ]:
# 1️⃣  Define your base path (adjust to your project)
base_path = "data/clean"
table1_path = os.path.join(base_path, "table_1")

In [ ]:
# 5️⃣  (Optional) Save each cleaned dataframe as a new CSV in a folder “clean_1”
output_folder = os.path.join(base_path, "clean_1")
os.makedirs(output_folder, exist_ok=True)

In [ ]:
for year in sorted(os.listdir(table1_path)):
    year_folder = os.path.join(table1_path, year)
    if os.path.isdir(year_folder):
        print(f"Cleaning year {year} ...")
        cleaned_dict = clean_table1_year(table1_path, year)

        # Save all monthly cleaned tables for this year
        for file, df in cleaned_dict.items():
            name = f"clean_1_{year}_{file.replace('.csv', '')}.csv"
            df.to_csv(os.path.join(output_folder, name), index=False)


In [ ]:
cleaned_dict.keys()

In [ ]:
cleaned_dict["ns-04-2022.csv"]

In [ ]:
import re
import os
import pandas as pd


def prepare_table_2(filepath: str, month_order_map: dict) -> pd.DataFrame:
    """
    
    """
    df = pd.read_csv(filepath)
    filename = os.path.basename(filepath)

    # 2. Drop unused columns
    df = df.drop(columns=["wr", "sectores_economicos"], errors="ignore")

    # 3. Create short economic sector labels
    SECTOR_MAP = {
        "agriculture and livestock": "agriculture",
        "fishing": "fishing",
        "mining and fuel": "mining",
        "manufacturing": "manufacturing",
        "electricity and water": "electricity",
        "construction": "construction",
        "commerce": "commerce",
        "other services": "services",
        "gdp": "gdp"
    }
    df["economic_sector"] = df["economic_sectors"].map(SECTOR_MAP)

    # 4. Filter valid sectors
    df = df[df["economic_sector"].notna()].copy()

    # 5. Create 'vintage_id'
    df["vintage_id"] = (
        df["economic_sector"] + "_" + df["year"].astype(str) + "_" + df["month"].astype(str)
    )

    # 6. Keep only yyyy_x columns where x ∈ {1, 2, 3, 4, year}
    pattern = re.compile(r"^\d{4}_(1|2|3|4|year)$")
    cols_to_keep = ["vintage_id"] + [c for c in df.columns if pattern.match(c)]
    df = df[cols_to_keep]

    # 7. Transpose and convert to tidy long format
    df_t = df.set_index("vintage_id").T.reset_index().rename(columns={"index": "target_period"})
    
    # Convert 'target_period' values:
    # - yyyy_(1|2|3|4) → yyyyq(1|2|3|4)
    # - yyyy_year → yyyy
    df_t["target_period"] = (
        df_t["target_period"]
        .str.replace(r"^(\d{4})_(\d)$", r"\1q\2", regex=True)  # quarterly pattern
        .str.replace(r"^(\d{4})_year$", r"\1", regex=True)     # annual pattern
    )

    return df_t

In [ ]:
def clean_table2_year(folder_path: str, year: str) -> dict:
    """Clean all monthly files for a given year and return dict of cleaned DataFrames."""
    year_folder = os.path.join(folder_path, year)
    month_order_map = build_month_order_map(year_folder)
    cleaned = {}
    for file in sorted(month_order_map.keys()):
        cleaned[file] = clean_table_2_file(os.path.join(year_folder, file), month_order_map)
    return cleaned

In [ ]:
# 1️⃣  Define your base path (adjust to your project)
base_path = "data/clean"
table2_path = os.path.join(base_path, "table_2")

In [ ]:
# 5️⃣  (Optional) Save each cleaned dataframe as a new CSV in a folder “clean_1”
output_folder = os.path.join(base_path, "clean_2")
os.makedirs(output_folder, exist_ok=True)

In [ ]:
for year in sorted(os.listdir(table2_path)):
    year_folder = os.path.join(table2_path, year)
    if os.path.isdir(year_folder):
        print(f"Cleaning year {year} ...")
        cleaned_dict_2 = clean_table2_year(table2_path, year)

        # Save all monthly cleaned tables for this year
        for file, df in cleaned_dict.items():
            name = f"clean_2_{year}_{file.replace('.csv', '')}.csv"
            df.to_csv(os.path.join(output_folder, name), index=False)


In [ ]:
cleaned_dict_2.keys()

In [ ]:
cleaned_dict_2["ns-12-2022.csv"]